# Decision Tree

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = {
    "Outlook": ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain'],
    "Temperature": ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild'],
    "Humidity": ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal'],
    "Windy": [False, True, False, False, False, True, True, False, False, False],
    "Play": ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes']
}

In [ ]:
df = pd.DataFrame(data)
df

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Rain,Mild,High,False,Yes
4,Rain,Cool,Normal,False,Yes
5,Rain,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Sunny,Mild,High,False,No
8,Sunny,Cool,Normal,False,Yes
9,Rain,Mild,Normal,False,Yes


In [ ]:
encoding = {}

for i in ['Outlook','Temperature','Humidity','Windy','Play']:
  encoding[i] = dict(enumerate(df[i].astype('category').cat.categories))
  df[i] = df[i].astype('category').cat.codes

inv_encoding = {i: {v:k for k,v in enc.items()} for i,enc in encoding.items()}

In [ ]:
# featutes and target
x = df.drop('Play', axis = 1).values
y = df['Play'].values

x,y

(array([[2, 1, 0, 0],
        [2, 1, 0, 1],
        [0, 1, 0, 0],
        [1, 2, 0, 0],
        [1, 0, 1, 0],
        [1, 0, 1, 1],
        [0, 0, 1, 1],
        [2, 2, 0, 0],
        [2, 0, 1, 0],
        [1, 2, 1, 0]], dtype=int8),
 array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1], dtype=int8))

In [ ]:
def gini_impurity(y):
  classes,counts = np.unique(y, return_counts=True)
  probability = counts/counts.sum()
  return 1-np.sum(probability**2)

In [ ]:
def split(x,y,feature_index,threshold):
  # all the rows in x, in the column of feature index.
  left_mask = x[:, feature_index] <= threshold
  right_mask = x[:, feature_index] > threshold
  return x[left_mask], y[left_mask], x[right_mask], y[right_mask]

In [ ]:
x.shape

(10, 4)

In [ ]:
x

array([[2, 1, 0, 0],
       [2, 1, 0, 1],
       [0, 1, 0, 0],
       [1, 2, 0, 0],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [0, 0, 1, 1],
       [2, 2, 0, 0],
       [2, 0, 1, 0],
       [1, 2, 1, 0]], dtype=int8)

In [ ]:
def best_split(x,y):
  best_feature = None
  best_threshold = None
  best_gini = np.inf # actually the worst...no information gain

# feature index -> column
# looks at eact unique values in each column
# for each unique t in thresholds:
  # the data will be splitted in to lef tor right.
  # based on gini impurity index.

  for feature_index in range(x.shape[1]):
    thresholds = np.unique(x[:, feature_index])
    for t in thresholds:
      x_left, y_left, x_right, y_right = split(x,y,feature_index, t)
      if (len(y_left) == 0) or (len(y_right) == 0):
        continue

      gini_left = gini_impurity(y_left)
      gini_right = gini_impurity(y_right)
      total_gini = ((len(y_left) * gini_left) + (len(y_right) * gini_right)) / len(y)

      if total_gini < best_gini:
        best_gini = total_gini
        best_feature = feature_index
        best_threshold = t

  return best_feature,best_threshold

In [ ]:
class node:
  def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

In [ ]:
def build_tree(x,y,depth = 0):

    # pure case : all labels in y is same
    # we return the leaf node containing the class label
    if(len(np.unique(y))==1):
      return node(value = y[0])

      # the best split
    feature, threshold = best_split(x,y)

      # no split
    if feature is None:
      majority = np.bincount(y).argmax()  # returns the frequency of all integers in y, and returns the index value of max frequency
      return node(value = majority) # it returns a prediction based on the most common label

        # EG : [0,0,1,1,1,1]
        # np.bincount(y): counts how many times each class appears
        # [2, 4] (means class 0 appeared 2 times, class 1 appeared 4 times)
        # .argmax(): returns the index of the highest count -> 1.


      # the branches
    x_left, y_left, x_right, y_right = split(x,y,feature,threshold)
        # Split the data into left and right branches
        # Divides the dataset based on the selected feature and threshold:
        # Left: values ≤ threshold
        # Right: values > threshold

      # left and right nodes...by recursion
    left_node = build_tree(x_left,y_left,depth + 1)
    right_node = build_tree(x_right,y_right,depth+1)

    return node(feature=feature, threshold=threshold, left=left_node, right=right_node)

In [ ]:
# Predicts the class label for one single input x by walking through the tree starting from the node (the root).

def predict_single(node,input):
  while node.value is None:   # keep going till leaf node.
                              # node.value will only be filled at a leaf (like class 0 or class 1).
    if input[node.feature] <= node.threshold:     # if the current input’s feature value is ≤ threshold → go left, else go right.
      node = node.left
    else:
      node = node.right
  return node.value     # (0/1)

In [ ]:
# For each row input in dataset x
# Collect all predicted values into a NumPy array...by calling the predict_single fn.

def predict_all(node,x):
  return np.array([predict_single(node,input) for input in x])

In [ ]:
# Build the tree
tree = build_tree(x, y)

# Predict on training data
y_pred = predict_all(tree, x)

# Accuracy
accuracy = np.sum(y_pred == y) / len(y)
print("Training Accuracy:", accuracy)

Training Accuracy: 1.0


In [ ]:
# user input